In [82]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
from functools import partial
import matplotlib

%matplotlib inline
import seaborn as sns
import category_encoders as ce

In [83]:
description = pd.read_csv(('HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv('application_train.csv')
previous_application = pd.read_csv('previous_application.csv')

In [84]:
description.tail(2)

,Unnamed: 0,Table,Row,Description,Special
217,220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN
218,221,installments_payments.csv,AMT_PAYMENT,What the client actually paid on previous cred...,NaN


In [85]:
application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
application.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [88]:
features = pd.DataFrame({'SK_ID_CURR': application['SK_ID_CURR']})

common_columns = [col for col in application.columns if col in previous_application.columns]
application_common = application[common_columns]
merged_tables = previous_application[common_columns + ['DAYS_DECISION']].merge(application_common, on='SK_ID_CURR',
                                                                               how='right')
merged_tables.head()
# features.head()
# common_columns
# application_common.head()
# merged_tables.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_x,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,NAME_TYPE_SUITE_x,WEEKDAY_APPR_PROCESS_START_x,HOUR_APPR_PROCESS_START_x,DAYS_DECISION,NAME_CONTRACT_TYPE_y,AMT_CREDIT_y,AMT_ANNUITY_y,AMT_GOODS_PRICE_y,NAME_TYPE_SUITE_y,WEEKDAY_APPR_PROCESS_START_y,HOUR_APPR_PROCESS_START_y
0,271877,Consumer loans,17145.0,1730.430,17145.0,NaN,SATURDAY,15.0,-73.0,Cash loans,533668.5,25803.0,477000.0,Unaccompanied,TUESDAY,17
1,271877,Consumer loans,1754721.0,68258.655,1800000.0,NaN,SATURDAY,18.0,-472.0,Cash loans,533668.5,25803.0,477000.0,Unaccompanied,TUESDAY,17
2,271877,Consumer loans,119848.5,12417.390,108400.5,NaN,SUNDAY,14.0,-548.0,Cash loans,533668.5,25803.0,477000.0,Unaccompanied,TUESDAY,17
3,108129,Cash loans,679671.0,25188.615,607500.0,Unaccompanied,THURSDAY,11.0,-164.0,Revolving loans,135000.0,6750.0,135000.0,Family,SUNDAY,10
4,108129,Cash loans,512370.0,21709.125,450000.0,NaN,WEDNESDAY,9.0,-515.0,Revolving loans,135000.0,6750.0,135000.0,Family,SUNDAY,10


In [89]:
o = application_common[application_common['SK_ID_CURR'] == 108129 ]
o

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START
6979,108129,Revolving loans,135000.0,6750.0,135000.0,Family,SUNDAY,10


In [90]:
merged_sorted = merged_tables.sort_values(['SK_ID_CURR', 'DAYS_DECISION'])
merged_sorted.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_x,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,NAME_TYPE_SUITE_x,WEEKDAY_APPR_PROCESS_START_x,HOUR_APPR_PROCESS_START_x,DAYS_DECISION,NAME_CONTRACT_TYPE_y,AMT_CREDIT_y,AMT_ANNUITY_y,AMT_GOODS_PRICE_y,NAME_TYPE_SUITE_y,WEEKDAY_APPR_PROCESS_START_y,HOUR_APPR_PROCESS_START_y
1345901,100002,Consumer loans,179055.0,9251.775,179055.0,NaN,SATURDAY,9.0,-606.0,Cash loans,406597.5,24700.5,351000.0,Unaccompanied,WEDNESDAY,10
1241278,100003,Consumer loans,68053.5,6737.310,68809.5,Family,SATURDAY,15.0,-2341.0,Cash loans,1293502.5,35698.5,1129500.0,Family,MONDAY,11
1241277,100003,Consumer loans,348637.5,64567.665,337500.0,Family,SUNDAY,17.0,-828.0,Cash loans,1293502.5,35698.5,1129500.0,Family,MONDAY,11
1241276,100003,Cash loans,1035882.0,98356.995,900000.0,Unaccompanied,FRIDAY,12.0,-746.0,Cash loans,1293502.5,35698.5,1129500.0,Family,MONDAY,11
1354064,100004,Consumer loans,20106.0,5357.250,24282.0,Unaccompanied,FRIDAY,5.0,-815.0,Revolving loans,135000.0,6750.0,135000.0,Unaccompanied,MONDAY,9


In [91]:
merged_sorted['annuity_diff'] = merged_sorted['AMT_ANNUITY_y'] - merged_sorted['AMT_ANNUITY_x']
merged_sorted['annuity_ratio'] = merged_sorted['AMT_ANNUITY_y'] / merged_sorted['AMT_ANNUITY_x']
merged_sorted['credit_diff'] = merged_sorted['AMT_CREDIT_y'] - merged_sorted['AMT_CREDIT_x']
merged_sorted['credit_ratio'] = merged_sorted['AMT_CREDIT_y'] / merged_sorted['AMT_CREDIT_x']
merged_sorted['annuity_diff'].head()

1345901    15448.725
1241278    28961.190
1241277   -28869.165
1241276   -62658.495
1354064     1392.750
Name: annuity_diff, dtype: float64

In [92]:
merged_sorted['the_same_contract_type'] = (
    merged_sorted['NAME_CONTRACT_TYPE_x'] == merged_sorted['NAME_CONTRACT_TYPE_y']).astype(int)
merged_sorted['the_same_weekday'] = (merged_sorted['WEEKDAY_APPR_PROCESS_START_x'] == merged_sorted['WEEKDAY_APPR_PROCESS_START_y']).astype(int)
merged_sorted['hour_diff'] = merged_sorted['HOUR_APPR_PROCESS_START_x'] - merged_sorted['HOUR_APPR_PROCESS_START_y']
merged_sorted['the_same_type_suite'] = (merged_sorted['NAME_TYPE_SUITE_x'] == merged_sorted['NAME_TYPE_SUITE_y']
                                       ).astype(int)
merged_sorted['the_same_type_suite'][merged_sorted['NAME_TYPE_SUITE_x'].isnull()] = 1

C:\Users\Umesh.k\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [93]:
merged_sorted['the_same_contract_type'].head()

1345901    0
1241278    0
1241277    0
1241276    1
1354064    0
Name: the_same_contract_type, dtype: int32

In [94]:
merged_sorted['NAME_CONTRACT_TYPE_x'].head()

1345901    Consumer loans
1241278    Consumer loans
1241277    Consumer loans
1241276        Cash loans
1354064    Consumer loans
Name: NAME_CONTRACT_TYPE_x, dtype: object

In [95]:
merged_sorted['NAME_CONTRACT_TYPE_y'].head()

1345901         Cash loans
1241278         Cash loans
1241277         Cash loans
1241276         Cash loans
1354064    Revolving loans
Name: NAME_CONTRACT_TYPE_y, dtype: object

In [96]:
merged_sorted['the_same_type_suite'].head()

1345901    1
1241278    1
1241277    1
1241276    0
1354064    1
Name: the_same_type_suite, dtype: int32

In [97]:
merged_sorted['NAME_TYPE_SUITE_x'].isnull().head() 

1345901     True
1241278    False
1241277    False
1241276    False
1354064    False
Name: NAME_TYPE_SUITE_x, dtype: bool

In [98]:
merged_sorted['NAME_TYPE_SUITE_y'].isnull().head() 

1345901    False
1241278    False
1241277    False
1241276    False
1354064    False
Name: NAME_TYPE_SUITE_y, dtype: bool

In [99]:
merged_sorted['the_same_type_suite'][merged_sorted['NAME_TYPE_SUITE_x'].isnull()].head()

1345901    1
491790     1
491788     1
491785     1
491789     1
Name: the_same_type_suite, dtype: int32

In [100]:
features.head()

,SK_ID_CURR
0,100002
1,100003
2,100004
3,100006
4,100007


In [101]:
# copy = merged_sorted.copy()
# copy.shape

In [102]:
# merged_sorted = copy.copy()
# merged_sorted= merged_sorted[['SK_ID_CURR','annuity_diff', 'annuity_ratio']]

# df1 = merged_sorted[merged_sorted["SK_ID_CURR"].isin([' 100002', '100003','100007'])]
# print(df1.head())

In [103]:
def _get_last_k_applications_feature_name(feature_name, number, suffix):
    return 'application_previous_application_{}_last_{}_applications_{}'.format(feature_name, number, suffix)


def get_last_k_credits_features(merged_sorted, numbers_of_applications):
    features = pd.DataFrame({'SK_ID_CURR': merged_sorted['SK_ID_CURR'].unique()})
    feature_list = ['annuity_diff', 'annuity_ratio', 'credit_diff', 'credit_ratio', 'the_same_contract_type',
                        'the_same_type_suite', 'the_same_weekday', 'hour_diff']

    for number in numbers_of_applications:
        table_tail = merged_sorted.groupby('SK_ID_CURR').tail(number)
        tail_groupby = table_tail.groupby('SK_ID_CURR')
        g = tail_groupby[feature_list].agg('mean')

        g = g.rename(axis='columns', mapper=partial(_get_last_k_applications_feature_name, number=number,
                                        suffix='mean')).reset_index()

        features = features.merge(g, how='left', on=['SK_ID_CURR'])
    return features

In [104]:
# print(numbers_of_applications)
# for number in [1]:
#         table_tail = merged_sorted.groupby('SK_ID_CURR').tail(number)
#         print(table_tail.shape)
#         table_tail_s = merged_sorted.groupby('SK_ID_CURR').tail(number)
#         print(table_tail.shape)
        
# print(table_tail.head())
# print(table_tail.head())

In [105]:
# gk123 = merged_sorted.groupby('SK_ID_CURR').tail(1)
# gk123
# gk1234= gk123.groupby('SK_ID_CURR')
# gk1234.head()
# # merged_sorted.tail(1)
# # agg

In [106]:
# merged_sorted['SK_ID_CURR'].unique()
# # g.columns

In [107]:
# merged_sorted.head()

In [108]:
# print(features.tail())
# # merged_sorted.head()
# numbers_of_applications=[1,3,5,10]
# merged_sorted.head()

In [109]:
g = get_last_k_credits_features(merged_sorted, numbers_of_applications=[1,3,5,10])
features = features.merge(g, on=['SK_ID_CURR'], how='left')

In [110]:
display(features.head(20))



,SK_ID_CURR,application_previous_application_annuity_diff_last_1_applications_mean,application_previous_application_annuity_ratio_last_1_applications_mean,application_previous_application_credit_diff_last_1_applications_mean,application_previous_application_credit_ratio_last_1_applications_mean,application_previous_application_the_same_contract_type_last_1_applications_mean,application_previous_application_the_same_type_suite_last_1_applications_mean,application_previous_application_the_same_weekday_last_1_applications_mean,application_previous_application_hour_diff_last_1_applications_mean,application_previous_application_annuity_diff_last_3_applications_mean,...,application_previous_application_the_same_weekday_last_5_applications_mean,application_previous_application_hour_diff_last_5_applications_mean,application_previous_application_annuity_diff_last_10_applications_mean,application_previous_application_annuity_ratio_last_10_applications_mean,application_previous_application_credit_diff_last_10_applications_mean,application_previous_application_credit_ratio_last_10_applications_mean,application_previous_application_the_same_contract_type_last_10_applications_mean,application_previous_application_the_same_type_suite_last_10_applications_mean,application_previous_application_the_same_weekday_last_10_applications_mean,application_previous_application_hour_diff_last_10_applications_mean
0,100002,15448.725,2.669812,227542.500,2.270797,0,1,0,-1.0,15448.7250,...,0.00,-1.000000,15448.725000,2.669812,2.275425e+05,2.270797,0.000000,1.000000,0.000000,-1.000000
1,100003,-62658.495,0.362948,257620.500,1.248697,1,0,0,1.0,-20855.4900,...,0.00,3.666667,-20855.490000,2.071487,8.093115e+05,7.988668,0.333333,0.666667,0.000000,3.666667
2,100004,1392.750,1.259975,114894.000,6.714414,0,1,0,-4.0,1392.7500,...,0.00,-4.000000,1392.750000,1.259975,1.148940e+05,6.714414,0.000000,1.000000,0.000000,-4.000000
3,100006,-3009.600,0.907952,-593932.500,0.344890,1,1,0,-2.0,-3009.6000,...,0.00,-2.000000,6035.325000,3.008889,2.098700e+04,inf,0.555556,0.888889,0.000000,-2.333333
4,100007,5827.860,1.363386,238711.500,1.870294,1,1,0,0.0,6679.4250,...,0.20,1.200000,9586.695000,3.887344,3.463612e+05,10.121560,0.666667,0.666667,0.166667,1.333333
5,100008,NaN,NaN,490495.500,inf,1,1,0,2.0,5919.7950,...,0.00,-4.000000,11677.803750,2.080781,3.277278e+05,inf,0.400000,0.800000,0.000000,-4.000000
6,100009,30882.330,3.964134,1472094.000,17.609058,0,1,0,1.0,32387.4450,...,0.40,-2.000000,31249.587857,4.410582,1.490588e+06,26.093714,0.000000,0.571429,0.285714,-2.285714
7,100010,14611.590,1.532038,1269189.000,5.866317,0,0,0,0.0,14611.5900,...,0.00,0.000000,14611.590000,1.532038,1.269189e+06,5.866317,0.000000,0.000000,0.000000,0.000000
8,100011,NaN,NaN,1019610.000,inf,1,0,1,-5.0,13678.8750,...,0.25,-2.250000,15523.305000,2.384665,7.577696e+05,inf,0.500000,0.500000,0.250000,-2.250000
9,100012,NaN,NaN,405000.000,inf,0,1,0,5.0,9914.8050,...,0.25,4.500000,12355.845000,3.556152,3.308805e+05,inf,0.000000,0.750000,0.250000,4.500000


In [111]:
features.to_csv("df_application previous",index=False)

In [112]:
from IPython.display import display, HTML
# df2 = g[g["SK_ID_CURR"].isin([' 100002', '100003','100007'])]
# display(df2.head())
# display(df1.head())
# print(df2.shape)
# print(df1.shape)

In [113]:
# print((28961.190-28869.165-62658.495)/3)

In [124]:
# g.head()

In [123]:
# g.shape

In [116]:
# X = application.merge(features,
#                                 left_on=['SK_ID_CURR'],
#                                 right_on=['SK_ID_CURR'],
#                                 how='left',
#                                 validate='one_to_one')

In [117]:
# engineered_numerical_columns = list(features.columns)
# engineered_numerical_columns.remove('SK_ID_CURR')
# X = X[engineered_numerical_columns + ['TARGET']]
# X_corr = abs(X.corr())

In [118]:
# len(engineered_numerical_columns)

In [119]:
# X_corr.sort_values('TARGET', ascending=False)['TARGET']

In [120]:
# X.shape

In [121]:
# X.dtypes

In [122]:
# X.head()